In [44]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd

In [19]:
df = pd.read_csv('creditcard.csv')

In [68]:
tf.reset_default_graph()

def get_pipe(dataset,batch_size=3):
    data = tf.data.Dataset.from_tensor_slices( tf.cast(dataset,dtype=tf.float32) ) 
    data = data.shuffle(3000).batch(batch_size)
    batch = data.make_one_shot_iterator().get_next()
    return batch[:,:-1], tf.cast(batch[:,-1],tf.int32)

def get_model(input):
    
    with tf.variable_scope('Layer1'):
        w1 = tf.get_variable('W1',shape=[30,20], initializer=tf.random_normal_initializer)
        b1 = tf.get_variable('B1',shape=[20], initializer=tf.random_normal_initializer)
        l1 = tf.nn.relu( tf.matmul(input,w1) + b1 )
        
    with tf.variable_scope('Layer2'):
        w2 = tf.get_variable('W2',shape=[20,5], initializer=tf.random_normal_initializer)
        b2 = tf.get_variable('B2',shape=[5], initializer=tf.random_normal_initializer)
        l2 = tf.nn.relu( tf.matmul(l1,w2) + b2 )
        
    return l2


def get_loss(logits,labels):

    # Cost function: Cross Entropy
    #cost = -tf.reduce_sum(label * tf.log(pred))
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits( labels=labels, logits=logits, name='cross_entropy')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy_mean')
    
    ################################################
    ## Record the loss value (cross_entropy_mean) ##
    ################################################
    
    tf.summary.scalar('cross_entropy_mean', cross_entropy_mean)
    reg_loss = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables() if v.name[0] != 'B']) * 0.001
    
    return cross_entropy_mean + reg_loss


def train(loss,learning_rate=0.001):
    
    return tf.train.AdamOptimizer(learning_rate).minimize(loss)

    

In [69]:
data,label = get_pipe(df.as_matrix())

In [70]:
logits = get_model(data)
loss = get_loss(logits,label)
train_op = train(loss)
merged = tf.summary.merge_all()

In [71]:
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

In [72]:

with tf.Session() as sess:
    if not os.path.exists('./tensorlog') :
        os.makedirs('./tensorlog')
        
    
    train_writer = tf.summary.FileWriter('./tensorlog',graph = sess.graph)
    
    sess.run(init)
    
    for i in range(10000):
        
        summary ,_ = sess.run([merged ,train_op])
        train_writer.add_summary(summary, i)
        
        if(i+100)%10 == 0:
            error = sess.run(loss)
            print(i,error)
    

0 2329.8293
10 826.0789
20 900.8084
30 359.0131
40 329.7238
50 1.9787418
60 1.9763488
70 1.4486873
80 1.9718066
90 1.9696119
100 1.4309748
110 1.9653302
120 1.9632385
130 1.4246986
140 1.959148
150 1.4206686
160 1.418698
170 1.9531989
180 1.9512737
190 1.9493877
200 1.9475306
210 1.9457006
220 1.943897
230 1.9423766
240 1.9409779
250 1.939589
260 1.9382124
270 1.9368491
280 1.9354999
290 1.9341645
300 1.9328291
310 1.9315069
320 1.9302107
330 1.9289323
340 1.9276686
350 1.9264184
360 1.9251813
370 1.9239731
380 1.9227817
390 1.9215958
400 1.9204206
410 1.9192566
420 1.9181039
430 1.3804834
440 1.9158329
450 1.9147142
460 1.9137192
470 1.9128616
480 1.9119776
490 1.9110882
500 1.9102018
510 1.9093212
520 1.371968
530 1.9075806
540 1.3702415
550 1.9058843
560 1.9050894
570 1.904276
580 1.9034599
590 1.9026471
600 1.9018399
610 1.9010388
620 1.900244
630 1.8994555
640 1.8986735
650 1.8978372
660 1.8970318
670 1.8962562
680 1.895495
690 1.8947428
700 1.8939976
710 1.8932585
720 1.8925252
7

5760 1.7209648
5770 1.720633
5780 1.7203007
5790 1.7199682
5800 1.7196356
5810 1.7193031
5820 1.7189709
5830 1.7186388
5840 1.7183069
5850 1.7179751
5860 1.7176436
5870 1.7173122
5880 1.716981
5890 1.71665
5900 1.7163192
5910 1.7159886
5920 1.7156583
5930 1.7153282
5940 1.7149982
5950 1.7146685
5960 1.7143391
5970 1.7140099
5980 1.7136809
5990 1.7133522
6000 1.7130237
6010 1.7126955
6020 1.7123675
6030 1.71204
6040 1.7117125
6050 1.7113854
6060 1.7110585
6070 1.710732
6080 1.7104057
6090 1.7100798
6100 1.7097541
6110 1.7094288
6120 1.7091037
6130 1.708779
6140 1.7084546
6150 1.7081306
6160 1.7078068
6170 1.7074834
6180 1.7071604
6190 1.7068377
6200 1.7065153
6210 1.7061933
6220 1.7058717
6230 1.7055504
6240 1.7052296
6250 1.7049091
6260 1.704589
6270 1.7042692
6280 1.7039499
6290 1.7036309
6300 1.7033124
6310 1.7029942
6320 1.7026765
6330 1.7023592
6340 1.7020423
6350 1.701726
6360 1.7014098
6370 1.7010943
6380 1.7007792
6390 1.7004645
6400 1.7001503
6410 1.6998364
6420 1.6995231
6430 